In [23]:
import pandas as pd
import numpy as np
from flowprintOptimal.sekigo.core.flowRepresentation import PacketFlowRepressentation
from flowprintOptimal.sekigo.flowUtils.commons import getIATFromTimeStamps
from flowprintOptimal.sekigo.flowUtils.commons import saveFlows
import datetime

In [43]:

class DeploymentProcessor:
    def __init__(self,csv_path, filter_out_classes = ["Music"]):
        self.df = pd.read_csv(csv_path)
        self.df.Timestamp = pd.to_datetime(self.df.Timestamp)
        self.filter_out_classes = filter_out_classes



    

    @staticmethod
    def processLength(length : float):
        if length <= 1500:
            return -2 + length/1500
        
        val =  -1 + np.log10(length - 1500)
        if val > 3:
            return 3
        return val
        
    
    def getPacketRepsFromDf(self,df):
        packet_reps = []

        for _,mini_df in df.groupby(by = "FlowId"):
            mini_df.sort_values(by= "Timestamp", inplace= True)
            IAT = getIATFromTimeStamps(timestamps= mini_df.Timestamp.values)
            directions = mini_df.Direction.values.astype(np.int8).tolist()
            lengths = mini_df.Length.apply(DeploymentProcessor.processLength).values.tolist()
            class_type = mini_df.iloc[0].Type

            if class_type in self.filter_out_classes:
                continue

            packet_rep = PacketFlowRepressentation(lengths= lengths, directions= directions, inter_arrival_times= IAT,class_type= class_type)
            packet_reps.append(packet_rep)
        
        return packet_reps


    def getPacketReps(self):
        return self.getPacketRepsFromDf(df = self.df)


In [44]:
deployment_processor = DeploymentProcessor(csv_path= "data/deployment/truncated_final.csv")

In [45]:
packet_reps = deployment_processor.getPacketReps()

In [46]:
for packet_rep in packet_reps:
    if pd.Series(packet_rep.inter_arrival_times).isna().sum() > 0:
        print(packet_rep)

In [47]:
saveFlows(path= "data/deployment/deployment_packet_rep.json", flows= packet_reps)